Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# From 3D Segmentation to Immersive Visualization: A Complete Workflow for Mesh Conversion, USD Export, and NVIDIA Omniverse Integration
In this tutorial, we’ll cover:

- Utilizing 3D Segmentation Results: How to extract and prepare segmentation data from VISTA-3D or MAISI for mesh conversion.
- Converting to Mesh Format: Step-by-step instructions on transforming segmentation results into mesh models.
- Exporting to USD: A guide to exporting meshes as Universal Scene Description (USD) files, optimized for Omniverse workflows.
- Visualizing in NVIDIA Omniverse: Instructions on importing USD files into Omniverse for high-quality 3D visualization and manipulation.
This end-to-end process enables efficient, high-quality visualization in NVIDIA Omniverse from raw segmentation data.

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import vtk" || pip install -q vtk
!python -c "import pxr" || pip install -q usd-core
!python -c "import trimesh" || pip install -q trimesh
!apt update
!apt install -y libgl1-mesa-glx

## Setup imports

In [1]:
import os
import tempfile
import numpy as np
from utility import convert_to_mesh, convert_mesh_to_usd

import vtk
import vtkmodules
from vtkmodules.vtkRenderingCore import vtkRenderWindow, vtkRenderer
from ipyvtklink.viewer import ViewInteractiveWidget

from monai.bundle.scripts import create_workflow, download
from monai.transforms import LoadImaged, SaveImage, Compose, BorderPadd, SqueezeDimd

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/Data


## Generate segmentation from VISTA-3D


In [4]:
download(name="vista3d", bundle_dir=".")

2024-12-03 07:24:15,516 - INFO - --- input summary of monai.bundle.scripts.download ---
2024-12-03 07:24:15,517 - INFO - > name: 'vista3d'
2024-12-03 07:24:15,517 - INFO - > bundle_dir: '.'
2024-12-03 07:24:15,518 - INFO - > source: 'monaihosting'
2024-12-03 07:24:15,518 - INFO - > remove_prefix: 'monai_'
2024-12-03 07:24:15,518 - INFO - > progress: True
2024-12-03 07:24:15,519 - INFO - ---


2024-12-03 07:24:16,246 - INFO - Expected md5 is None, skip md5 check for file vista3d_v0.5.7.zip.
2024-12-03 07:24:16,247 - INFO - File exists: vista3d_v0.5.7.zip, skipped downloading.
2024-12-03 07:24:16,247 - INFO - Writing into directory: ..


In [4]:
bundle_root = os.path.join("/workspace/Code/tutorials/modules/omniverse", "vista3d")
input_dict = {"image": "/workspace/Code/tutorials/modules/totalSegmentator_mergedLabel_samples/imagesTr/s0001.nii.gz"}
workflow = create_workflow(
    config_file=os.path.join(bundle_root, "configs/inference.json"),
    workflow_type="inference",
    bundle_root=bundle_root,
    input_dict=input_dict,
)
workflow.run()

2024-12-03 07:38:37,279 - INFO - Setting logging properties based on config: /workspace/Code/tutorials/modules/omniverse/vista3d/configs/logging.conf.
2024-12-03 07:38:38,118 - py.warnings - WARNING - The torch.distributed is either unavailable and uninitiated when RankFilter is instantiated.
If torch.distributed is used, please ensure that the RankFilter() is called
after torch.distributed.init_process_group() in the script.


2024-12-03 07:38:38,119 - py.warnings - WARNING - You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer b

[None]

## Convert NII to Mesh and Save as OBJ

In [2]:
# 17 groupings that cover 101 segments/regions out of 140
labels = {
    "Liver": 1,
    "Spleen": 3,
    "Pancreas": 4,
    "Heart": 115,
    "Body": 200,
    "Gallbladder": 10,
    "Stomach": 12,
    "Small_bowel": 19,
    "Colon": 62,
    "Kidney": {"right_kidney": 5, "left_kidney": 14},
    "Veins": {
        "aorta": 6,
        "inferior_vena_cava": 7,
        "portal_vein_and_splenic_vein": 17,
        "left_iliac_artery": 58,
        "right_iliac_artery": 59,
        "left_iliac_vena": 60,
        "right_iliac_vena": 61,
        "pulmonary_vein": 119,
        "left_subclavian_artery": 123,
        "right_subclavian_artery": 124,
        "superior_vena_cava": 125,
        "brachiocephalic_trunk": 109,
        "left_brachiocephalic_vein": 110,
        "right_brachiocephalic_vein": 111,
        "left_common_carotid_artery": 112,
        "right_common_carotid_artery": 113,
    },
    "Lungs": {
        "left_lung_upper_lobe": 28,
        "left_lung_lower_lobe": 29,
        "right_lung_upper_lobe": 30,
        "right_lung_middle_lobe": 31,
        "right_lung_lower_lobe": 32,
    },
    "Spine": {
        # "vertebrae_L6": 131,
        "vertebrae_L5": 33,
        "vertebrae_L4": 34,
        "vertebrae_L3": 35,
        "vertebrae_L2": 36,
        "vertebrae_L1": 37,
        "vertebrae_T12": 38,
        "vertebrae_T11": 39,
        "vertebrae_T10": 40,
        "vertebrae_T9": 41,
        "vertebrae_T8": 42,
        "vertebrae_T7": 43,
        "vertebrae_T6": 44,
        "vertebrae_T5": 45,
        "vertebrae_T4": 46,
        "vertebrae_T3": 47,
        "vertebrae_T2": 48,
        "vertebrae_T1": 49,
        "vertebrae_C7": 50,
        "vertebrae_C6": 51,
        "vertebrae_C5": 52,
        "vertebrae_C4": 53,
        "vertebrae_C3": 54,
        "vertebrae_C2": 55,
        "vertebrae_C1": 56,
        "sacrum": 97,
        "vertebrae_S1": 127,
    },
    "Ribs": {
        "left_rib_1": 63,
        "left_rib_2": 64,
        "left_rib_3": 65,
        "left_rib_4": 66,
        "left_rib_5": 67,
        "left_rib_6": 68,
        "left_rib_7": 69,
        "left_rib_8": 70,
        "left_rib_9": 71,
        "left_rib_10": 72,
        "left_rib_11": 73,
        "left_rib_12": 74,
        "right_rib_1": 75,
        "right_rib_2": 76,
        "right_rib_3": 77,
        "right_rib_4": 78,
        "right_rib_5": 79,
        "right_rib_6": 80,
        "right_rib_7": 81,
        "right_rib_8": 82,
        "right_rib_9": 83,
        "right_rib_10": 84,
        "right_rib_11": 85,
        "right_rib_12": 86,
        "costal_cartilages": 114,
        "sternum": 122,
    },
    "Shoulders": {"left_scapula": 89, "right_scapula": 90, "left_clavicula": 91, "right_clavicula": 92},
    "Hips": {"left_hip": 95, "right_hip": 96},
    "Back_muscles": {
        "left_gluteus_maximus": 98,
        "right_gluteus_maximus": 99,
        "left_gluteus_medius": 100,
        "right_gluteus_medius": 101,
        "left_gluteus_minimus": 102,
        "right_gluteus_minimus": 103,
        "left_autochthon": 104,
        "right_autochthon": 105,
        "left_iliopsoas": 106,
        "right_iliopsoas": 107,
    },
}


def nii_to_obj(input_nii_path, output_nii_path, output_obj_path):
    if not os.path.exists(output_nii_path):
        os.makedirs(output_nii_path)
    pre_trans = Compose(
        [
            LoadImaged(keys="label", ensure_channel_first=True),
            BorderPadd(keys="label", spatial_border=2),
            SqueezeDimd(keys="label", dim=0),
        ]
    )
    orig_seg = pre_trans({"label": input_nii_path})["label"]
    all_organ = np.zeros_like(orig_seg, dtype=np.uint8)

    save_trans = SaveImage(output_ext="nii.gz", output_dtype=np.uint8)
    #     save_trans.set_options(write_kwargs = {"compression":True})
    for j, (organ_name, label_val) in enumerate(labels.items(), start=1):
        single_organ = np.zeros_like(orig_seg, dtype=np.uint8)
        print(f"Assigning index {j} to label {organ_name}")
        if isinstance(label_val, dict):
            for _, i in label_val.items():
                all_organ[orig_seg == i] = j
                single_organ[orig_seg == i] = j
        else:
            all_organ[orig_seg == label_val] = j
            single_organ[orig_seg == label_val] = j
        organ_filename = os.path.join(output_nii_path, organ_name)
        save_trans(single_organ[None], meta_data=orig_seg.meta, filename=organ_filename)
        convert_to_mesh(
            f"{organ_filename}.nii.gz",
            output_obj_path,
            f"{organ_name}.obj",
            label_value=j,
            smoothing_factor=0.5,
            reduction_ratio=0.0,
        )

    all_organ_filename = os.path.join(output_nii_path, "all_organs")
    save_trans(all_organ[None], meta_data=orig_seg.meta, filename=all_organ_filename)
    convert_to_mesh(
        f"{all_organ_filename}.nii.gz",
        output_obj_path,
        "all_organs.gltf",
        label_value=list(range(1, 18)),
        smoothing_factor=0.6,
        reduction_ratio=0.0,
    )
    print(f"Saved whole segmentation {all_organ_filename}")


input_nii_path = "/workspace/Data/maisi_ct_generative/datasets/IntegrationTest-AbdomenCT.nii.gz"
output_nii_path = "/workspace/Data/maisi_ct_generative/datasets/monai/nii"
output_obj_path = "/workspace/Data/maisi_ct_generative/datasets/monai/obj"
out = nii_to_obj(input_nii_path, output_nii_path, output_obj_path)

Assigning index 1 to label Liver
2024-12-03 09:13:00,689 INFO image_writer.py:197 - writing: /workspace/Data/maisi_ct_generative/datasets/monai/nii/Liver.nii.gz
Mesh successfully exported to /workspace/Data/maisi_ct_generative/datasets/monai/obj/Liver.obj
Assigning index 2 to label Spleen
2024-12-03 09:13:07,086 INFO image_writer.py:197 - writing: /workspace/Data/maisi_ct_generative/datasets/monai/nii/Spleen.nii.gz
Mesh successfully exported to /workspace/Data/maisi_ct_generative/datasets/monai/obj/Spleen.obj
Assigning index 3 to label Pancreas
2024-12-03 09:13:10,982 INFO image_writer.py:197 - writing: /workspace/Data/maisi_ct_generative/datasets/monai/nii/Pancreas.nii.gz
Mesh successfully exported to /workspace/Data/maisi_ct_generative/datasets/monai/obj/Pancreas.obj
Assigning index 4 to label Heart
2024-12-03 09:13:14,755 INFO image_writer.py:197 - writing: /workspace/Data/maisi_ct_generative/datasets/monai/nii/Heart.nii.gz
Mesh successfully exported to /workspace/Data/maisi_ct_gene

## Visualize one single organ mesh

In [1]:
# Step 1: Read the mesh
reader = vtk.vtkOBJReader()
reader.SetFileName(
    "/workspace/Data/maisi_ct_generative/datasets/monai/obj/Heart.obj"
)  # Provide the path to your STL file
reader.Update()

# Step 2: Create a mapper
mapper = vtkmodules.vtkRenderingCore.vtkPolyDataMapper()
mapper.SetInputConnection(reader.GetOutputPort())

print(f"Number of Points: {mapper.GetInput().GetNumberOfPoints()}")
print(f"Number of Cells: {mapper.GetInput().GetNumberOfCells()}")
# Step 3: Create an actor
actor = vtkmodules.vtkRenderingCore.vtkActor()
actor.SetMapper(mapper)

# Step 4: Set up the renderer
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.2, 0.4)  # Background color
renderer.ResetCamera()

# Step 5: Set up the render window
render_window = vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.Render()
render_window.SetSize(800, 600)

# Step 6: Use ipyvtklink for notebook rendering
widget = ViewInteractiveWidget(render_window)
widget

Number of Points: 2874
Number of Cells: 5744


ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=800)

![Liver](Liver.png)

![all organs](all_organs.png)

## Convert OBJ to USD

In [4]:
obj_filename = "/workspace/Data/maisi_ct_generative/datasets/monai/obj/all_organs.gltf"
usd_filename = "/workspace/Data/maisi_ct_generative/datasets/monai/obj/all_organs.usd"

convert_mesh_to_usd(obj_filename, usd_filename)

USD file successfully exported to /workspace/Data/maisi_ct_generative/datasets/monai/obj/all_organs.usd
